<a href="https://colab.research.google.com/github/xehau/xehau_lessons/blob/master/csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# tf.data を使って CSV をロードする

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルでは、CSV データを `tf.data.Dataset` にロードする手法の例を示します。

このチュートリアルで使われているデータはタイタニック号の乗客リストから取られたものです。乗客が生き残る可能性を、年齢、性別、チケットの等級、そして乗客が一人で旅行しているか否かといった特性から予測することを試みます。

## 設定

In [ ]:
import functools

import numpy as np
import tensorflow as tf

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [ ]:
# numpy の値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

## データのロード

それではいつものように、扱っている CSV ファイルの先頭を見てみましょう。

In [ ]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


[pandasでデータを読み込んでから](pandas_dataframe.ipynb) Numpy の array を TensorFlow に渡すというのは可能です。もし、大規模なデータに対応するためにスケールアップしたり、データの読み込み処理を [TensorFlow や tf.data](../../guide/data.ipynb) に統合されたものにしたい場合、`tf.data.experimental.make_csv_dataset` 関数が利用できます。

The only column you need to identify explicitly is the one with the value that the model is intended to predict. 

In [ ]:
LABELS = [0, 1]
LABEL_COLUMN = 'survived'

コンストラクタの引数の値が揃ったので、ファイルから CSV データを読み込みデータセットを作ることにしましょう。

（完全なドキュメントは、`tf.data.experimental.make_csv_dataset` を参照してください）

In [ ]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

データセットを構成する要素は、(複数のサンプル, 複数のラベル)の形のタプルとして表されるバッチです。サンプル中のデータは（行ベースのテンソルではなく）列ベースのテンソルとして構成され、それぞれはバッチサイズ（このケースでは5個）の要素が含まれます。

実際に見てみましょう。

In [ ]:
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'female' b'male' b'female']
age                 : [48.  26.  42.  17.  32.5]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [2 0 0 0 0]
fare                : [ 65.      7.925 227.525   7.125  13.   ]
class               : [b'Second' b'Third' b'First' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


上で見たように、この CSV の列には名前がついています。Dataset のコンストラクターはこれらの列名を自動的に抽出します。一行目に列名が記されていない CSV を扱う場合には、列名のリストを `make_csv_dataset` 関数の `column_names` 引数に渡してください。

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'male' b'female' b'female']
age                 : [28. 16. 25. 22. 23.]
n_siblings_spouses  : [0 1 1 0 0]
parch               : [0 3 0 1 0]
fare                : [ 7.75  34.375  7.775 55.     7.925]
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'y']


このサンプルでは利用できるすべての列を使います。もし、データセットから特定の列を除外したい場合には、利用したい列名のリストを作成し、コンストラクタのオプショナルな引数である `select_columns` にそのリストを渡してください。

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [34. 36. 18. 21. 34.]
n_siblings_spouses  : [1 0 1 0 0]
class               : [b'Second' b'Second' b'Third' b'First' b'Second']
deck                : [b'unknown' b'D' b'unknown' b'D' b'unknown']
alone               : [b'n' b'y' b'n' b'n' b'n']


## データの前処理

CSV ファイルに含まれるデータには様々な型のものがありえます。データをモデルに入力する前に、典型的なケースでは、様々な型を含んだデータを固定長のベクトルに変換する必要があります。

TensorFlow には典型的なデータ変換を行う `tf.feature_column` が組み込まれています。詳細は[このチュートリアル](../keras/feature_columns)を参照してください。

また、任意のツール (たとえば、[nltk](https://www.nltk.org/) や [sklearn](https://scikit-learn.org/stable/) ) を使って前処理を行い、前処理した結果を TensorFlow にわたすこともできます。

モデルの内部で前処理を行う最大の利点は、モデルをエクスポートしたときにその中に前処理が含まれていることです。この方法を取れば、生データをそのままモデルに入力することができます。

### 連続データ

すでにデータが適切に数値型になっている場合、モデルに渡す前にそのデータをベクトル形式にできます。

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28.  28.  17.  40.5 28. ]
n_siblings_spouses  : [1. 0. 0. 0. 0.]
parch               : [0. 0. 0. 2. 0.]
fare                : [24.     7.896  8.663 14.5   12.35 ]


In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

次のサンプルは、数値型の列をベクトルに変換するシンプルな関数の例です。

In [ ]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

この関数をデータセットのそれぞれの要素に適用します。

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.     1.     0.    82.171]
 [28.     0.     0.     7.75 ]
 [30.5    0.     0.     8.05 ]
 [23.     0.     0.     7.55 ]
 [14.     0.     0.     7.854]]

[1 1 0 1 0]


様々な型を含んだデータセットを利用する場合、シンプルな数値型のフィールドだけを分離したくなるかもしれません。`tf.feature_column` はそのような処理を実現できますが、いくらかのオーバーヘッドを発生させるため、それが本当に必要になる場合以外ではできるだけ避けるべきでしょう。様々な型を含んだデータセットに話を戻しましょう。

In [ ]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [70.5 28.  36.  26.  28. ]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 7.75   7.896 10.5    7.896  7.796]
class               : [b'Third' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [ ]:
example_batch, labels_batch = next(iter(temp_dataset))

数値型の特徴量を選んで、それらをベクトル化して単一の列に変換する、より一般的な前処理器を作成しましょう。

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [ ]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
class               : [b'Second' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'F' b'unknown' b'G' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']
numeric             : [[ 0.83   0.     2.    29.   ]
 [25.     0.     0.     7.65 ]
 [43.     0.     0.     8.05 ]
 [ 4.     1.     1.    16.7  ]
 [40.     0.     0.     7.896]]


In [ ]:
example_batch, labels_batch = next(iter(packed_train_data))

#### データの正規化

連続値をとるデータは常に正規化されるべきでしょう。

In [ ]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


数値型の列を作成しましょう。`tf.feature_columns.numeric_column` では `normalizer_fn` 引数で正則化のための関数を受け取ります。また、渡された関数はそれぞれのバッチに対して実行されます。

[`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial) を用いて関数を部分適用し、`normalize_numeric_data` の引数 `MEAN` と `STD` を固定しておきましょう。

In [ ]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f0e8c0e7440>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

モデルの訓練時には、数値データのブロックを選び出して利用できるように、作成した特徴量の列を入力に含めましょう。

In [ ]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   ,  0.   ],
       [28.   ,  0.   ,  0.   , 56.496],
       [ 7.   ,  4.   ,  1.   , 39.688],
       [ 3.   ,  4.   ,  2.   , 31.388],
       [23.   ,  0.   ,  0.   ,  7.854]], dtype=float32)>

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.63 ],
       [-0.13 , -0.474, -0.479,  0.405],
       [-1.809,  3.001,  0.782,  0.097],
       [-2.128,  3.001,  2.043, -0.055],
       [-0.53 , -0.474, -0.479, -0.486]], dtype=float32)

ここで用いられた、平均を用いた正規化を利用するためには、それぞれの列の平均値を事前に知っている必要があります。

### カテゴリデータ

この CSV データ中のいくつかの列はカテゴリ列です。つまり、その中身は、限られた選択肢の中のひとつである必要があります。

`tf.feature_column` API を用いて、それぞれのカテゴリ列のためのコレクションを `tf.feature_column.indicator_column` で生成しましょう。

In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [ ]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [ ]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


モデルを構築する際、これは入力レイヤーで行われるデータ処理の一部になります。

### 一体化した前処理レイヤー

2つの特徴量列のコレクションを結合し、`tf.keras.layers.DenseFeatures` に入力して、両方のデータ型を読み込んで前処理する入力レイヤーを作成しましょう。

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     1.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
 -0.479 -0.63   1.     0.   ]


## モデルの構築

`preprocessing_layer` から始まる `tf.keras.Sequential` を構築しましょう。

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## 訓練、評価、そして予測

これでモデルをインスタンス化し、訓練することができます。

In [ ]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [ ]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 1s 2ms/step - loss: 0.5584 - accuracy: 0.7020
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4349 - accuracy: 0.8068
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.8228
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3882 - accuracy: 0.8320
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8250
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3677 - accuracy: 0.8176
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3452 - accuracy: 0.8382
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3531 - accuracy: 0.8466
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3685 - accurac

モデルの訓練が終わったら、`test_data` データセットでの正解率をチェックできます。

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4488 - accuracy: 0.8409


Test Loss 0.44881781935691833, Test Accuracy 0.8409090638160706


単一のバッチ、または、バッチからなるデータセットのラベルを推論する場合には、`tf.keras.Model.predict` を使います。

In [ ]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: 115.22%  | Actual outcome:  DIED
Predicted survival: -274.22%  | Actual outcome:  DIED
Predicted survival: -75.44%  | Actual outcome:  SURVIVED
Predicted survival: -374.96%  | Actual outcome:  DIED
Predicted survival: -59.06%  | Actual outcome:  SURVIVED
